*Author: Shannon O'Hare

Project: Titanic Kaggle Competition

Date: 2020/05/22*

In [ ]:
# Load packages
library('ggplot2') # visualization
library('ggthemes') # visualization
library('scales') # visualization
library('dplyr') # data manipulation
library('mice') # imputation
library('randomForest') # classification algorithm

In [ ]:
train <- read.csv('../input/titanic/train.csv', stringsAsFactors = F)
test  <- read.csv('../input/titanic/test.csv', stringsAsFactors = F)

full  <- bind_rows(train, test) # bind training & test data
head(full)

In [ ]:
summary(full)

In [ ]:
# Grab title from passenger names
full$Title <- gsub('(.*, )|(\\..*)', '', full$Name)
?gsub

In [ ]:
# Show title counts by sex
table(full$Sex, full$Title)

In [ ]:
# Titles with very low cell counts to be combined to "rare" level
rare_title <- c('Dona', 'Lady', 'the Countess','Capt', 'Col', 'Don', 
                'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer')

# Also reassign mlle, ms, and mme accordingly
full$Title[full$Title == 'Mlle']        <- 'Miss' 
full$Title[full$Title == 'Ms']          <- 'Miss'
full$Title[full$Title == 'Mme']         <- 'Mrs' 
full$Title[full$Title %in% rare_title]  <- 'Rare Title'

# Show title counts by sex again
table(full$Sex, full$Title)

In [ ]:
# Finally, grab surname from passenger name
full$Surname <- sapply(full$Name,  
                      function(x) strsplit(x, split = '[,.]')[[1]][1])
                       
# print(full$Surname)
                       

Now that we’ve taken care of splitting passenger name into some new variables, we can take it a step further and make some new family variables. First we’re going to make a family size variable based on number of siblings/spouse(s) (maybe someone has more than one spouse?) and number of children/parents.

In [ ]:
# Create a family size variable including the passenger themselves
full$Fsize <- full$SibSp + full$Parch + 1

# Create a family variable 
full$Family <- paste(full$Surname, full$Fsize, sep='_')

What does our family size variable look like? To help us understand how it may relate to survival, let’s plot it among the training data.

In [ ]:
# Use ggplot2 to visualize the relationship between family size & survival
ggplot(full[1:891,], aes(x = Fsize, fill = factor(Survived))) +
  geom_bar(stat='count', position='dodge') +
  scale_x_continuous(breaks=c(1:11)) +
  labs(x = 'Family Size') +
  theme_few()

#aes includes the axis
#geom bar puts the data on the graph in the form of a bar
#Labs( Title = "...", subtitle = "...") = labels
#scale_x_continuous for continuous data - break nicely

Survival = 0 is did not survive. 
A lot of non-survivors for family size of 1 (single people)
We can collapse this variable into *three levels* which will be helpful since there are comparatively fewer large families. Let’s create a *discrete family size* variable.

In [ ]:
# Discrete family size
full$FsizeD[full$Fsize == 1] <- 'singleton'
full$FsizeD[full$Fsize <= 4 & full$Fsize > 1] <- 'small'
full$FsizeD[full$Fsize > 4] <- 'large'

# Show family size by survival using a mosaic plot
mosaicplot(table(full$FsizeD, full$Survived), main='Family Size by Survival', shade=TRUE)

The mosaic plot shows that we preserve our rule that there’s a survival penalty among singletons and large families, but a benefit for passengers in small families. I want to do something further with our age variable, but 263 rows have missing age values, so we will have to wait until after we address missingness

Potentially useful information in the **passenger cabin** variable including about their **deck**...

In [ ]:
# This variable appears to have a lot of missing values
full$Cabin[1:28]

In [ ]:
# The first character is the deck. For example:
strsplit(full$Cabin[2], NULL)[[1]]

In [ ]:
# Create a Deck variable. Get passenger deck A - F:
full$Deck <-factor(sapply(full$Cabin, function(x) strsplit(x, NULL)[[1]][1]))
                          
# Data isn't great - abandon

 **Imputation**
 
 1st: Sensible value imputation
 
 2nd: Prediction
 
 
 1. Sensible Value Imputation

In [ ]:
# Replace missing data with sensible made-up data (median,mode etc) or use prediction... 

# Passengers 62 and 830 are missing Embarkment
full[c(62, 830), 'Embarked']

In [ ]:
# Get rid of our missing passenger IDs
embark_fare <- full %>%
  filter(PassengerId != 62 & PassengerId != 830)

#Filter passID where it's not 62 or 830

# We see that they paid $ 80 and $ NA respectively and their classes are 1 and NA . 
# So from where did they embark?

# Use ggplot2 to visualize embarkment, passenger class, & median fare
ggplot(embark_fare, aes(x = Embarked, y = Fare, fill = factor(Pclass))) +
  geom_boxplot() +
  geom_hline(aes(yintercept=80), 
    colour='red', linetype='dashed', lwd=1) +
  scale_y_continuous(labels=dollar_format()) +
  theme_few()

The median fare for a first class passenger departing from Charbourg (‘C’) coincides nicely with the $80 paid by our embarkment-deficient passengers. I think we can safely replace the NA values with ‘C’.

In [ ]:
# Since their fare was $80 for 1st class, they most likely embarked from 'C'
full$Embarked[c(62, 830)] <- 'C'

We’re close to fixing the handful of NA values here and there. Passenger on row 1044 has an NA Fare value.

In [ ]:
# Show row 1044
full[1044, ]

# This is a third class passenger who departed from Southampton (‘S’). 
# Visualise Fares among all others sharing their class and embarkment (n = 494).

In [ ]:
ggplot(full[full$Pclass == '3' & full$Embarked == 'S', ], 
  aes(x = Fare)) +
  geom_density(fill = '#99d6ff', alpha=0.4) + 
  geom_vline(aes(xintercept=median(Fare, na.rm=T)),
    colour='red', linetype='dashed', lwd=1) +
  scale_x_continuous(labels=dollar_format()) + theme_few()

# From this visualisation, it seems quite reasonable to replace the NA Fare value with 
# median for their class and embarkment which is $8.05.

In [ ]:
# Replace missing fare value with median fare for class/embarkment
full$Fare[1044] <- median(full[full$Pclass == '3' & full$Embarked == 'S', ]$Fare, na.rm = TRUE)

2. Prediction Imputation
* There are quite a few missing Age values in our data.
* Create a model predicting ages based on other variables.

In [ ]:
# Show number of missing Age values
sum(is.na(full$Age))

In [ ]:
# Make variables factors into factors
factor_vars <- c('PassengerId','Pclass','Sex','Embarked',
                 'Title','Surname','Family','FsizeD')

full[factor_vars] <- lapply(full[factor_vars], function(x) as.factor(x))

# Set a random seed
set.seed(129)

# Perform mice imputation, excluding certain less-than-useful variables:
mice_mod <- mice(full[, !names(full) %in% c('PassengerId','Name','Ticket','Cabin','Family','Surname','Survived')], method='rf') 

In [ ]:
# Save the complete output 
mice_output <- complete(mice_mod)

In [ ]:
# Compare the results we get with the original distribution of passenger ages to ensure that nothing has gone completely awry.

# Plot age distributions
par(mfrow=c(1,2))
hist(full$Age, freq=F, main='Age: Original Data', 
  col='darkgreen', ylim=c(0,0.04))
hist(mice_output$Age, freq=F, main='Age: MICE Output', 
  col='lightgreen', ylim=c(0,0.04))

In [ ]:
# Things look good, let’s replace our age vector in the original data with the output from the mice model.

# Replace Age variable from the mice model.
full$Age <- mice_output$Age

# Show new number of missing Age values
sum(is.na(full$Age))

Now that we know everyone’s age, we can create a couple of new age-dependent variables: Child and Mother. A child will simply be someone under 18 years of age and a mother is a passenger who is 1) female, 2) is over 18, 3) has more than 0 children

In [ ]:
# First we'll look at the relationship between age & survival
ggplot(full[1:891,], aes(Age, fill = factor(Survived))) + 
  geom_histogram() + 
  # I include Sex since we know (a priori) it's a significant predictor
  facet_grid(.~Sex) + 
  theme_few()

In [ ]:
# Create the column child, and indicate whether child or adult
full$Child[full$Age < 18] <- 'Child'
full$Child[full$Age >= 18] <- 'Adult'

# Show counts
table(full$Child, full$Survived)

In [ ]:
# Adding Mother variable
full$Mother <- 'Not Mother'
full$Mother[full$Sex == 'female' & full$Parch > 0 & full$Age > 18] <- 'Mother'
# Parch = Number of parents/children aboard

# Show counts
table(full$Mother, full$Survived)

In [ ]:
# Finish by factorizing our two new factor variables
full$Child  <- factor(full$Child)
full$Mother <- factor(full$Mother)

In [ ]:
# All of the variables we care about should be taken care of and there should be no missing data. I’m going to double check just to be sure:

md.pattern(full)


**Prediction**

In [ ]:
# Split the data back into a train set and a test set
train <- full[1:891,]
test <- full[892:1309,]

In [ ]:
# Set a random seed
set.seed(754)

# Build the model (note: not all possible variables are used)
rf_model <- randomForest(factor(Survived) ~ Pclass + Sex + Age + SibSp + Parch + 
                                            Fare + Embarked + Title + 
                                            FsizeD + Child + Mother,
                                            data = train)

# Show model error
plot(rf_model, ylim=c(0,0.36))
legend('topright', colnames(rf_model$err.rate), col=1:3, fill=1:3)

# The black line shows the overall error rate which falls below 20%. 
# The red and green lines show the error rate for ‘died’ and ‘survived’ respectively. 
# We can see that right now we’re much more successful predicting death than we are survival.


In [ ]:
# Look at relative variable importance by plotting the mean decrease in Gini calculated across all trees.

# Get importance
importance    <- importance(rf_model)
varImportance <- data.frame(Variables = row.names(importance), 
                            Importance = round(importance[ ,'MeanDecreaseGini'],2))

# Create a rank variable based on importance
rankImportance <- varImportance %>%
  mutate(Rank = paste0('#',dense_rank(desc(Importance))))

# Use ggplot2 to visualize the relative importance of variables
ggplot(rankImportance, aes(x = reorder(Variables, Importance), 
    y = Importance, fill = Importance)) +
  geom_bar(stat='identity') + 
  geom_text(aes(x = Variables, y = 0.5, label = Rank),
    hjust=0, vjust=0.55, size = 4, colour = 'red') +
  labs(x = 'Variables') +
  coord_flip() + 
  theme_few()

Title variable has the highest relative importance out of all of our predictor variables.

In [ ]:
### Prediction :

# Predict using the test set
prediction <- predict(rf_model, test)

# Save the solution to a dataframe with two columns: PassengerId and Survived (prediction)
solution <- data.frame(PassengerID = test$PassengerId, Survived = prediction)

# Write the solution to file
write.csv(solution, file = 'rf_mod_Solution.csv', row.names = F)